# AA test    

_An A/A test is a variation of an A/B test, the peculiarity of which is that the original is compared with itself, as opposed to an A/B test, which compares samples before and after exposure._

## 0. Import Libraries

In [ ]:
import numpy as np
import pandas as pd

from hypex import AATest
from hypex.utils.tutorial_data_creation import create_test_data

pd.options.display.float_format = '{:,.2f}'.format

np.random.seed(42)  # needed to create example data

In [ ]:
def show_result(result):
    for k, v in result.items():
        print(k)
        display(v)
        print()

## 1. Create or upload your dataset
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 

In [ ]:
data = create_test_data(rs=52, na_step=10, nan_cols=['age', 'gender'])
data

## 2. AATest 

### 2.0 Initialize parameters
`info_col` used to define informative attributes that should NOT be part of testing, such as user_id and signup_month <br>

In [ ]:
info_cols = ['user_id', 'signup_month']
target = ['post_spends', 'pre_spends']

### 2.1 Simple AA-test
This is the easiest way to initialize and calculate metrics on a AA-test (default - on 2000 iterations)<br>Use it when you are clear about each attribute or if you don't have any additional task conditions (like grouping)    

You can also add some extra arguments to the process(): 
* plot_set - types of plot, that you want to show ("hist", "cumulative", "percentile")
* figsize - size of figure for plots 
* alpha - value to change the transparency of the histogram plot 
* bins - generic bin parameter that can be the name of a reference rule, the number of bins, or the breaks of the bins 
* title_size - size of title for plots

In [ ]:
experiment = AATest(info_cols=info_cols, target_fields=target)

In [ ]:
results = experiment.process(data, iterations=2000)

In [ ]:
show_result(results)

In [ ]:
results.keys()

`results` is a dictionary with dataframes as values.<br>
* 'split' - result of separation, column 'group' contains values 'test' and 'control'  
* 'resume' - summary of all results   
* 'aa_score' - score of T-test and Kolmogorov-Smirnov test 
* 'experiments' - is a table of results of experiments, which includes 
    - means of all targets in a and b samples, 
    - p_values of Student t-test and test Kolmogorova-Smirnova, 
    - and results of tests (did data on the random_state passes the uniform test)
* 'best_experiment_stat' - like previous point but only for the best experiment 
* 'split_stat' - metrics and statistics tests for result of split



In [ ]:
results['aa_score']

In [ ]:
results['resume']

### 2.2 Single experiment
To get stable results lets fix `random_state`

In [ ]:
random_state = 11

To perform single experiment you can use `sampling_metrics()`

In [ ]:
experiment = AATest(info_cols=info_cols, target_fields=target)
metrics, dict_of_datas = experiment.sampling_metrics(data=data, random_state=random_state).values()

The results contains the same info as in multisampling, but on one experiment

In [ ]:
metrics

In [ ]:
dict_of_datas[random_state]

In [ ]:
results = experiment.experiment_result_transform(pd.Series(metrics))

In [ ]:
results.keys()

In [ ]:
results['best_experiment_stat']

In [ ]:
results['best_split_stat']

### 2.3 AA-test with grouping

To perform experiment that separates samples by groups `group_col` can be used

In [ ]:
info_cols = ['user_id', 'signup_month']
target = ['post_spends', 'pre_spends']

group_cols = 'industry'

In [ ]:
experiment = AATest(info_cols=info_cols, target_fields=target, group_cols=group_cols)

In [ ]:
results = experiment.process(data=data, iterations=2000)

The result is in the same format as without groups

In this regime groups equally divided on each sample (test and control):

In [ ]:
results['split']['industry'].value_counts(normalize=True) * 100

In [ ]:
results['split'].groupby(['industry', 'group'])[['user_id']].count()

In [ ]:
show_result(results)

### 2.4 AA with optimize group 

_If you have many columns for grouping and don't know which colun or columns will make best result, you can use parametr `optimize_group=True`.
AA-Test will choose optimal number and names of group columns._

You can use `columns_labeling` to automatically name columns as target and group.

In [ ]:
experiment.columns_labeling(data)

In [ ]:
results = experiment.process(data=data, optimize_groups=True, iterations=2000)

In [ ]:
experiment.group_cols

In [ ]:
show_result(results)

### 2.5 AA test with quantization 

_If you want make one column as parameter for quantization, you may use `quant_field`._

In [ ]:
info_cols = ['user_id', 'signup_month']
target = ['post_spends', 'pre_spends']

group_cols = 'industry'
quant_field = 'gender'

In [ ]:
experiment = AATest(info_cols=info_cols, target_fields=target, group_cols=group_cols, quant_field=quant_field)

In [ ]:
result = experiment.process(data=data, iterations=2000)

In [ ]:
result['split'].groupby(['gender', 'industry', 'group'])['user_id'].count()


In [ ]:
show_result(result)

### 2.6 Unbalanced AA test

_If you want to perform AA test with unbalanced groups, you can use parametr `test_size` to define sizes of test group and control group_

In [ ]:
info_cols = ['user_id', 'signup_month']
target = ['post_spends', 'pre_spends']

group_cols = 'industry'

In [ ]:
experiment = AATest(info_cols=info_cols, target_fields=target, group_cols=group_cols)

In [ ]:
result = experiment.process(data=data, test_size=0.3, iterations=2000)

In [ ]:
result['split']['group'].value_counts(normalize=True)

In [ ]:
show_result(result)

## MDE    
this is the boundary value of the effect, for which it makes sense to introduce some changes. 

In [ ]:
info_cols = ['user_id', 'signup_month']
target = ['post_spends', 'pre_spends']

group_cols = 'industry'
mde_target = 'post_spends'

In [ ]:
experiment = AATest(info_cols=info_cols, target_fields=target, group_cols=group_cols)

Single experiment of data splitting for MDE calculation. 

_P.s. [None] is the number of random state. You can change it like sampling_metrics(data, random_state=42) and get result with [42] instead of [None]_ 

In [ ]:
splitted_data = experiment.sampling_metrics(data)['data_from_experiment'][None]
splitted_data

In [ ]:
splitted_data[mde_target].hist()

You can evaluate minimum detectable effect for your data. This will be the smallest true effect obtained from the changes, which the statistical criterion will be able to detect with confidence 

In [ ]:
mde = experiment.calc_mde(data=splitted_data, group_field="group", target_field=mde_target)
mde

You can also calculate the amount of data you need to have in order to determine the minimum effect of the test.

In [ ]:
experiment.calc_sample_size(data=splitted_data, target_field=mde_target, mde=5)

## Chi2 Test

In [ ]:
target = ['post_spends', 'pre_spends'] 
treated_field = 'treat'

In [ ]:
experiment = AATest(target_fields=target)

In [ ]:
experiment.calc_chi2(data, treated_field)